In [1]:
import cv2

In [2]:
from deepface import DeepFace
import numpy as np

In [3]:
import tensorflow as tf

In [4]:
import datetime,pygame,keyboard

pygame 2.5.1 (SDL 2.28.2, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
import xlwings as xw

In [6]:
#xlwings 
workbook = xw.Book('attendance.xlsx')
sheet_name = datetime.date.today().isoformat()
try:
    worksheet = workbook.sheets(sheet_name)
except:
    worksheet = workbook.sheets.add(sheet_name)

In [7]:
worksheet.range('A1').value = 'STUDENT'
worksheet.range('B1').value = 'DATE'
worksheet.range('C1').value = 'TIME'
students=[]

In [8]:
model_live = 'liveness.model'
model_live = tf.keras.models.load_model(model_live)

In [9]:
s=2
i=0
entered = False
al_entered = False

In [10]:
cap = cv2.VideoCapture(0)

#pygame
pygame.init()
pg_txt = (39,112,50)
pg_bg = (154,179,157)
font = cv2.FONT_HERSHEY_SIMPLEX
pygame_font = pygame.font.Font('freesansbold.ttf',32)

screen = pygame.display.set_mode((960,540))

bgimg = pygame.image.load('bg_img.png')
bgimg = pygame.transform.rotozoom(bgimg,0,1)

screen.blit(bgimg,(0,0))

#day today 
t0 = datetime.date.today()
t0 = t0.day

running = True

while running:
    moment = datetime.datetime.now()
    hr = moment.hour 
    mins = moment.minute 
    day = moment.day
    month = moment.month
    year = moment.year
    date = f"{day}-{month}-{year}"
    time = f"{hr}:{mins}"
        
    screen.blit(bgimg,(0,0))
        
    #check 
    
    if day!=t0:
        t0 = day
        worksheet = workbook.sheets.add(date)
        worksheet.range('A1').value = 'STUDENT'
        worksheet.range('B1').value = 'DATE'
        worksheet.range('C1').value = 'TIME'
        students=[]
        i=0
        entered =  False
        al_entered = False
        
    state,frame = cap.read()
    
    if state!=True:
        break 
    res = DeepFace.find(frame, db_path='./Deepface', enforce_detection = False, model_name='VGG-Face')
    if len(res[0]['identity'][0])>0:
        name = res[0]['identity'][0].split('/')[1].split('\\')[1]
        xmin = res[0]['source_x'][0]
        ymin = res[0]['source_y'][0]
        w = res[0]['source_w'][0]
        h = res[0]['source_h'][0]
        xmax = xmin+w
        ymax = ymin+h
        
        #liveness
        
        img = frame[ymin:ymax,xmin:xmax]
        img = cv2.resize(img,(32,32))
        # model accepts the liveness to (32,32) instead of the original one it will resize
        img = img.astype('float')/255.0 
        # to normalize we divide as all values in rgb range from 0-255 we are now converting to 0-1
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = np.expand_dims(img,axis=0)
        
        liveness = model_live.predict(img)
      
        liveness = liveness[0].argmax()
        
        if liveness == 1:
            cv2.rectangle(frame, (xmin,ymin), (xmax,ymax), (0,255,0),2)
            cv2.putText(frame,name,(xmin,ymin),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3,cv2.LINE_AA)
            cv2.putText(frame, str(liveness), (xmin,ymax+25),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2,cv2.LINE_AA )
            
            
            n_txt = pygame_font.render(name,True,pg_txt,pg_bg)
            n_textrect = n_txt.get_rect()
            n_textrect.center = (960-200,540//2)
            
            d_txt = pygame_font.render(date,True,pg_txt,pg_bg)
            d_textrect = d_txt.get_rect()
            d_textrect.center = (960-200,(540//2)+40)
            
            t_txt = pygame_font.render(time,True,pg_txt,pg_bg)
            t_textrect = t_txt.get_rect()
            t_textrect.center = (960-200,(540//2)+(80))
            
            screen.blit(n_txt, n_textrect)
            screen.blit(d_txt, d_textrect)
            screen.blit(t_txt, t_textrect)
            
            if keyboard.is_pressed('enter'):
                if name not in students:
                    worksheet.range(f'A{s}').value = name
                    worksheet.range(f'B{s}').value = date
                    worksheet.range(f'C{s}').value = time
                    students.append(name)
                    s+=1
                    entered = True
                else:
                    al_entered = True
                    
            if i<1 and entered==True:
                e_txt = pygame_font.render('entered',True,pg_txt,pg_bg)
                e_textrect = e_txt.get_rect()
                e_textrect.center = (960-200,(540//2)+120)
                screen.blit(e_txt,e_textrect)
                i+=1
            elif i<30 and al_entered==True:
                ae_txt = pygame_font.render('already entered',True,pg_txt,pg_bg)
                ae_textrect = ae_txt.get_rect()
                ae_textrect.center = (960-200,(540//2)+120)
                screen.blit(ae_txt,ae_textrect)
                i+=1
            else:
                entered = False
                al_entered = False
                i=0
            
            
        
 
    image = pygame.image.frombuffer(frame.tostring(), frame.shape[1::-1],'BGR')
    image = pygame.transform.rotozoom(image,0,0.8)

    size = image.get_size()
    rect_img = pygame.Surface(size, pygame.SRCALPHA)
    pygame.draw.rect(rect_img,(255,255,255),(0,0,*size), border_radius=10)

    image = image.copy().convert_alpha()
    image.blit(rect_img, (0,0), None, pygame.BLEND_RGB_MIN)
    screen.blit(image,(20,80))

    pygame.display.update()
    for event in  pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        
#     cv2.imshow('attendance', frame)
#     c = cv2.waitKey(1)
#     if c==ord('q'):
#         break 


cap.release()
pygame.quit()
#cv2.destroyAllWindows()

There are  7  representations found in  representations_vgg_face.pkl
find function lasts  2.219984292984009  seconds
1/1 [==============================] - 0s 113ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.23648428916931152  seconds
1/1 [==============================] - 0s 22ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.2307732105255127  seconds
1/1 [==============================] - 0s 17ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.22359752655029297  seconds
1/1 [==============================] - 0s 18ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.22754120826721191  seconds
1/1 [==============================] - 0s 17ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.2243638038635254  seconds
1/1 [========================

1/1 [==============================] - 0s 17ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.3823728561401367  seconds
1/1 [==============================] - 0s 16ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.3828146457672119  seconds
1/1 [==============================] - 0s 17ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.23265433311462402  seconds
1/1 [==============================] - 0s 18ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.2180767059326172  seconds
1/1 [==============================] - 0s 16ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.22036504745483398  seconds
1/1 [==============================] - 0s 17ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.2191

find function lasts  0.24764537811279297  seconds
1/1 [==============================] - 0s 29ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.266160249710083  seconds
1/1 [==============================] - 0s 19ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.41750383377075195  seconds
1/1 [==============================] - 0s 17ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.22651100158691406  seconds
1/1 [==============================] - 0s 18ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.2504253387451172  seconds
1/1 [==============================] - 0s 20ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.24644064903259277  seconds
1/1 [==============================] - 0s 24ms/step
There are  7  representations found in  repres

find function lasts  0.22798395156860352  seconds
1/1 [==============================] - 0s 18ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.23058056831359863  seconds
1/1 [==============================] - 0s 18ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.21111464500427246  seconds
1/1 [==============================] - 0s 16ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.2301955223083496  seconds
1/1 [==============================] - 0s 17ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.2247331142425537  seconds
1/1 [==============================] - 0s 18ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.2144618034362793  seconds
1/1 [==============================] - 0s 18ms/step
There are  7  representations found in  repres

find function lasts  0.2199246883392334  seconds
1/1 [==============================] - 0s 17ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.21944952011108398  seconds
1/1 [==============================] - 0s 16ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.22056818008422852  seconds
1/1 [==============================] - 0s 16ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.22153377532958984  seconds
1/1 [==============================] - 0s 17ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.21852564811706543  seconds
1/1 [==============================] - 0s 17ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.2294292449951172  seconds
1/1 [==============================] - 0s 17ms/step
There are  7  representations found in  repre

find function lasts  0.20508241653442383  seconds
1/1 [==============================] - 0s 18ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.23467040061950684  seconds
1/1 [==============================] - 0s 19ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.23572278022766113  seconds
1/1 [==============================] - 0s 17ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.3805270195007324  seconds
1/1 [==============================] - 0s 17ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.3890388011932373  seconds
1/1 [==============================] - 0s 17ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.5400617122650146  seconds
1/1 [==============================] - 0s 17ms/step
There are  7  representations found in  repres

find function lasts  0.39249229431152344  seconds
1/1 [==============================] - 0s 17ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.19930601119995117  seconds
1/1 [==============================] - 0s 16ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.5427894592285156  seconds
1/1 [==============================] - 0s 21ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.3747248649597168  seconds
1/1 [==============================] - 0s 17ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.38005614280700684  seconds
1/1 [==============================] - 0s 18ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.3907289505004883  seconds
1/1 [==============================] - 0s 28ms/step
There are  7  representations found in  repres

find function lasts  0.1890580654144287  seconds
1/1 [==============================] - 0s 16ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.18650221824645996  seconds
1/1 [==============================] - 0s 18ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.19355106353759766  seconds
1/1 [==============================] - 0s 17ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.18541550636291504  seconds
1/1 [==============================] - 0s 18ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.1852104663848877  seconds
1/1 [==============================] - 0s 16ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.18434858322143555  seconds
1/1 [==============================] - 0s 18ms/step
There are  7  representations found in  repre

find function lasts  0.3720717430114746  seconds
1/1 [==============================] - 0s 17ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.2161872386932373  seconds
1/1 [==============================] - 0s 17ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.22933554649353027  seconds
1/1 [==============================] - 0s 16ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.22690701484680176  seconds
1/1 [==============================] - 0s 18ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.3872959613800049  seconds
1/1 [==============================] - 0s 16ms/step
There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.21070098876953125  seconds
1/1 [==============================] - 0s 17ms/step
There are  7  representations found in  repres

In [12]:
model = DeepFace.find(img_path = 'testg.jpg', db_path = './Deepface', enforce_detection = False, model_name='VGG-Face')

There are  7  representations found in  representations_vgg_face.pkl
find function lasts  0.22574377059936523  seconds


In [13]:
model[0]

,identity,source_x,source_y,source_w,source_h,VGG-Face_cosine
0,./Deepface\ganesh/WhatsApp Image 2023-09-15 at...,0,0,576,1024,0.113137
1,./Deepface\ganesh/WhatsApp Image 2023-09-15 at...,0,0,576,1024,0.135928
2,./Deepface\akhila/WhatsApp Image 2023-09-15 at...,0,0,576,1024,0.313153
3,./Deepface\akhila/WhatsApp Image 2023-09-15 at...,0,0,576,1024,0.331407


# Liveness check
img = frame[ymin:ymax, xmin:xmax]: This line extracts a region of interest (ROI) from the frame image. The ymin, ymax, xmin, and xmax values likely represent the coordinates of a bounding box around a detected face. This line of code creates a sub-image (img) by cropping the region of the frame within the specified bounding box.

img = cv2.resize(img, (32, 32)): After extracting the ROI, this line resizes the img to a smaller size of 32x32 pixels. It appears that the liveness detection model being used requires this specific input size.

img = img.astype('float') / 255.0: This line converts the pixel values in img to floating-point numbers and then normalizes them by dividing each pixel value by 255.0. Normalization is a common preprocessing step in machine learning to scale the pixel values from the original range (0-255 for RGB images) to a normalized range between 0 and 1. This helps the neural network converge faster during training.

img = tf.keras.preprocessing.image.img_to_array(img): This line converts the img from a NumPy array to a TensorFlow tensor. The img_to_array function is often used for preparing images for deep learning models.

img = numpy.expand_dims(img, axis=0): Finally, this line expands the dimensions of the img tensor by adding an extra dimension at the beginning. This is typically done to make the image compatible with models that expect a batch of images as input. By adding an extra dimension at the beginning, the shape of the img tensor becomes (1, 32, 32, 3) (assuming it's a color image with 3 channels).

In summary, the code is preparing a cropped and resized face image (img) for input to a liveness detection model. It resizes, normalizes, and converts the image to a format suitable for feeding into the model.







In [ ]:
!pip install xlwings

In [ ]:
!pip install pygame

In [ ]:
!pip install keyboard